In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the Lice`nse is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

从2024年9月15日开始，您只能通过切换到Vertex AI Gemini提示和调整来定制分类、实体提取和情感分析模型。针对文本分类、实体提取和情感分析目标进行Vertex AI AutoML模型的训练或更新将不再可用。您可以继续使用现有的Vertex AI AutoML文本目标直到2025年6月15日。有关Gemini如何通过改进的提示功能提供增强用户体验的更多信息，请参阅[调整简介](https://cloud.google.com/vertex-ai/generative-ai/docs/models/tune-gemini-overview)。

# Vertex AI： 创建、训练和部署自动ML文本分类模型

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl-text-classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl-text-classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
 <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl/automl-text-classification.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>

## 概述

本笔记本将引导您了解在[Vertex AI](https://cloud.google.com/vertex-ai/docs/)上构建和使用AutoML文本分类模型的主要阶段。

了解有关[文本数据分类](https://cloud.google.com/vertex-ai/docs/training-overview#classification_for_text)的更多信息。

### 目标

在本教程中，您将学习如何使用AutoML来训练文本分类模型。

本教程使用以下谷歌云ML服务：

- AutoML训练
- 顶点AI模型资源

执行的步骤包括：

* 创建一个顶点AI数据集。
* 训练一个AutoML文本分类模型资源。
* 获取模型资源的评估指标。
* 创建一个端点资源。
* 将模型资源部署到端点资源。
* 进行在线预测。
* 进行批量预测。

数据集

在这个笔记本中，您将使用“Happy Moments”样本数据集来训练模型。最终模型将快乐时刻分类到反映幸福原因的类别中。

### 成本

本教程使用 Google Cloud 的计费组件:

* Vertex AI 训练和提供服务
* 云存储

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预计使用量生成成本估算。

### 安装

安装以下软件包以执行此笔记本。

In [ ]:
# install packages
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                    google-cloud-storage \
                                    jsonlines 

### 仅限Colab：取消对以下单元格的注释以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 开始之前

#### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
- 运行 `gcloud config list`
- 运行 `gcloud projects list`
- 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# set the project id
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。
了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

### 验证您的 Google Cloud 账户

根据您的 Jupyter 环境，您可能需要手动进行身份验证。请按照以下相关说明操作。

**1. Vertex AI Workbench** 
- 不需要做任何操作，因为您已经通过身份验证。

**2. 本地 JupyterLab 实例，请取消注释并运行。**

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务帐号或其他
- 在这里查看所有身份验证选项：[Google Cloud Platform Jupyter Notebook身份验证指南](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_authentication_guide.ipynb)

创建一个云存储桶

创建一个存储桶，用于存储中间产物，如数据集。

In [ ]:
BUCKET_NAME = f"your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

如果您的存储桶尚不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

导入库并定义常量

In [ ]:
import jsonlines
from google.cloud import aiplatform, storage
from google.cloud.aiplatform import jobs

### 初始化Vertex AI

为您的项目初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

创建数据集资源并导入您的数据

该笔记本使用“Happy Moments”数据集进行演示。您可以更改为符合数据准备要求的另一个文本分类数据集。

使用Python SDK，创建数据集并在一次调用中导入数据，如下所示。

创建和导入数据是一个长时间运行的操作。接下来的步骤可能需要一些时间。`create()` 方法会等待操作完成，并随着操作的进行输出语句。这些语句包含下一节中使用的数据集的完整名称。

**注意**：您可以在等待此操作完成时关闭笔记本。

In [ ]:
# Use a timestamp to ensure unique resources
src_uris = "gs://cloud-ml-data/NL-classification/happiness.csv"
display_name = "e2e-text-dataset-unique"

text_dataset = aiplatform.TextDataset.create(
    display_name=display_name,
    gcs_source=src_uris,
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.single_label_classification,
    sync=True,
)

## 训练您的文本分类模型

现在您可以开始训练您的模型。训练模型是一个两部分的过程：

1. **定义训练作业。** 在定义训练作业时，您必须提供显示名称和您想要的训练类型。
2. **运行训练作业。** 运行训练作业时，您需要提供一个数据集的引用用于训练。您还可以配置数据拆分的百分比。

您不需要指定[数据拆分](https://cloud.google.com/vertex-ai/docs/general/ml-use)。如果您不提供值，训练作业的默认设置为训练80%/ 测试10%/ 验证10%。

要训练您的模型，您可以像下面的代码片段中所示调用 `AutoMLTextTrainingJob.run()`。该方法将返回对您的新模型对象的引用。

与将数据导入数据集一样，训练模型可能需要相当长的时间。客户端库在训练管道操作处理时打印操作状态消息。在您可以获取您的新模型的资源名称和ID之前，必须等待训练过程完成，这些信息对于模型评估和模型部署是必需的。

**注意**: 在等待操作完成时，您可以关闭笔记本。

In [ ]:
# Define the training job
training_job_display_name = "e2e-text-training-job-unique"
job = aiplatform.AutoMLTextTrainingJob(
    display_name=training_job_display_name,
    prediction_type="classification",
    multi_label=False,
)

In [ ]:
model_display_name = "e2e-text-classification-model-unique"

# Run the training job
model = job.run(
    dataset=text_dataset,
    model_display_name=model_display_name,
    training_fraction_split=0.1,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    sync=True,
)

## 查看模型评估分数

在模型训练完成后，您可以使用`list_model_evaluations()`方法查看其评估分数。该方法返回每个评估切片的迭代器。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

## 部署您的文本分类模型

一旦您的模型完成训练，您必须将其部署到一个 _端点_，以从模型获得在线预测。当您将模型部署到一个端点时，在端点上会复制模型，并生成一个新的资源名称和显示名称。

您可以将多个模型部署到同一个端点，并在分配给端点的各个模型之间进行流量分割。然而，您必须一次只部署一个模型到端点。若要更改流量分割百分比，您必须在每次部署新模型时为第二个（和后续）模型分配新值。

以下代码块展示了如何部署一个模型。这段代码依赖于 Python SDK 来为部署创建一个新的端点。调用 `modely.deploy()` 返回一个端点对象的引用 - 您将需要这个引用来进行下一节中的在线预测。

In [ ]:
deployed_model_display_name = "e2e-deployed-text-classification-model-unique"

endpoint = model.deploy(
    deployed_model_display_name=deployed_model_display_name, sync=True
)

## 从您的模型获得在线预测

现在您已经有了端点，您可以从文本分类模型获得在线预测。要获得在线预测，您需要向端点发送一个预测请求。

In [ ]:
content = "I got a high score on my math final!"

response = endpoint.predict(instances=[{"content": content}])

for prediction_ in response.predictions:
    ids = prediction_["ids"]
    display_names = prediction_["displayNames"]
    confidence_scores = prediction_["confidences"]
    for count, id in enumerate(ids):
        print(f"Prediction ID: {id}")
        print(f"Prediction display name: {display_names[count]}")
        print(f"Prediction confidence score: {confidence_scores[count]}")

## 获取模型的批量预测

您可以从文本分类模型获得批量预测，而无需部署它。您必须首先将所有预测实例（预测输入）以 JSONL 格式编排并将 JSONL 文件存储在 Google Cloud 存储桶中。您还必须提供一个 Google Cloud 存储桶来保存您的预测输出。

首先，您必须以 JSONL 格式创建您的预测输入文件。JSONL 文档中的每一行需要如下格式：

```
{ "content": "gs://sourcebucket/datasets/texts/source_text.txt", "mimeType": "text/plain"}
```

JSON 结构中的 `content` 字段必须是指向另一个包含用于预测的文本输入的文档的 Google Cloud 存储 URI。
[查看更多信息，请参阅文档。](https://cloud.google.com/ai-platform-unified/docs/predictions/batch-predictions#text)

In [ ]:
instances = [
    "We hiked through the woods and up the hill to the ice caves",
    "My kitten is so cute",
]
input_file_name = "batch-prediction-input.jsonl"

对于批量预测，请提供以下内容：

+ 将所有的预测实例作为单独的文件存储在Google Cloud Storage上，文件格式为TXT。
+ 一个JSONL文件，列出所有预测实例的URI。
+ 一个Cloud Storage存储桶，用于保存批量预测输出结果。

在本教程中，以下步骤将创建一个新的存储桶，将单独的预测实例上传为文本文件到该存储桶，并创建包含预测实例URI的JSONL文件。

In [ ]:
# Instantiate the Storage client and create the new bucket
# from google.cloud import  storage
storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)
# Iterate over the prediction instances, creating a new TXT file
# for each.
input_file_data = []
for count, instance in enumerate(instances):
    instance_name = f"input_{count}.txt"
    instance_file_uri = f"{BUCKET_URI}/{instance_name}"
    # Add the data to store in the JSONL input file.
    tmp_data = {"content": instance_file_uri, "mimeType": "text/plain"}
    input_file_data.append(tmp_data)

    # Create the new instance file
    blob = bucket.blob(instance_name)
    blob.upload_from_string(instance)

input_str = "\n".join([str(d) for d in input_file_data])
file_blob = bucket.blob(f"{input_file_name}")
file_blob.upload_from_string(input_str)

现在您已经准备好带有预测实例的桶，您可以将批量预测https://storage.googleapis.com/upload/storage/v1/b/gs://vertex-ai-devaip-20220728004429/o?uploadType=multipartequest 发送到Vertex AI。当您向服务发送请求时，您必须提供JSONL文件的URI和输出桶，包括`gs://`协议。

使用Python SDK，您可以通过调用`Model.batch_predict()`来创建批量预测作业。

In [ ]:
job_display_name = "e2e-text-classification-batch-prediction-job"
# model = aiplatform.Model(model_name=model.name)
batch_prediction_job = model.batch_predict(
    job_display_name=job_display_name,
    gcs_source=f"{BUCKET_URI}/{input_file_name}",
    gcs_destination_prefix=f"{BUCKET_URI}/output",
    sync=True,
)
batch_prediction_job_name = batch_prediction_job.resource_name

一旦批量预测作业完成，Python SDK会以`projects/[PROJECT_ID]/locations/[LOCATION]/batchPredictionJobs/[BATCH_PREDICTION_JOB_ID]`的格式打印出批量预测作业的资源名称。您可以使用其ID查询Vertex AI服务的批量预测作业状态。

以下代码片段演示了如何创建`BatchPredictionJob`类的实例以查看其状态。请注意，您需要从Python SDK中打印出的完整资源名称用于此代码片段。

批量预测任务

In [ ]:
batch_job = jobs.BatchPredictionJob(batch_prediction_job_name)
print(f"Batch prediction job state: {str(batch_job.state)}")

批处理作业完成后，您可以在输出存储桶中查看作业的结果。您可能首先需要列出输出存储桶中的所有文件，以查找输出文件的URI。

In [ ]:
BUCKET_OUTPUT = f"{BUCKET_URI}/output"

! gsutil ls -a $BUCKET_OUTPUT

批量预测作业的输出应该存储在一个包含批量预测作业名称和创建时间戳的文件夹（或前缀）中。

例如，如果您的批量预测作业名称是`my-job`，您的存储桶名称是`my-bucket`，包含输出的文件夹的URI可能如下所示：

```
gs://my-bucket/output/prediction-my-job-2021-06-04T19:54:25.889262Z/
```

要读取批量预测结果，您必须将文件下载到本地并打开文件。下一个单元格将所有文件复制到本地文件夹`BUCKET_OUTPUT_FOLDER`中。

In [ ]:
import os

RESULTS_DIRECTORY = "prediction_results"
RESULTS_DIRECTORY_FULL = f"{RESULTS_DIRECTORY}/output"

# Create missing directories
os.makedirs(RESULTS_DIRECTORY, exist_ok=True)

# Get the Cloud Storage paths for each result
! gsutil -m cp -r $BUCKET_OUTPUT $RESULTS_DIRECTORY

# Get most recently modified directory
latest_directory = max(
    (
        os.path.join(RESULTS_DIRECTORY_FULL, d)
        for d in os.listdir(RESULTS_DIRECTORY_FULL)
    ),
    key=os.path.getmtime,
)

print(f"Local results folder: {latest_directory}")

## 复习结果

所有的结果文件都已经下载到本地，您可以打开它们并阅读结果。在本教程中，您将使用[`jsonlines`](https://jsonlines.readthedocs.io/en/latest/)库来读取输出结果。

以下代码单元打开 JSONL 输出文件，然后打印每个实例的预测结果。

In [ ]:
# Get downloaded results in directory
results_files = []
for dirpath, _, files in os.walk(latest_directory):
    for file in files:
        if file.find("predictions") >= 0:
            results_files.append(os.path.join(dirpath, file))


# Consolidate all the results into a list
results = []
for results_file in results_files:
    # Open each result
    with jsonlines.open(results_file) as reader:
        for result in reader.iter(type=dict, skip_invalid=True):
            instance = result["instance"]
            prediction = result["prediction"]
            print(f"\ninstance: {instance['content']}")
            for key, output in prediction.items():
                print(f"\n{key}: {output}")

清理

要清理本项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

* 数据集
* 训练作业
* 模型
* 端点
* 批量预测
* 批量预测存储桶

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

# Delete batch
batch_job.delete()

# Undeploy endpoint
endpoint.undeploy_all()

# `force` parameter ensures that models are undeployed before deletion
endpoint.delete()

# Delete model
model.delete()

# Delete text dataset
text_dataset.delete()

# Delete training job
job.delete()

## 下一步

完成本教程后，请查看以下文档页面，了解有关 Vertex AI 的更多信息：

* [准备文本训练数据](https://cloud.google.com/vertex-ai/docs/training-overview#text_data)
* [使用 API 训练 AutoML 模型](https://cloud.google.com/vertex-ai/docs/training-overview#automl)
* [评估 AutoML 模型](https://cloud.google.com/vertex-ai/docs/training-overview#automl)
* [使用 Vertex AI API 部署模型](https://cloud.google.com/vertex-ai/docs/predictions/overview#model_deployment)
* [从 AutoML 模型获取在线预测](https://cloud.google.com/vertex-ai/docs/predictions/overview#model_deployment)
* [获取批量预测](https://cloud.google.com/vertex-ai/docs/predictions/overview#batch_predictions)